In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [2]:
!kaggle competitions download -c llm-detect-ai-generated-text

 72% 1.00M/1.38M [00:00<00:00, 1.41MB/s]
100% 1.38M/1.38M [00:00<00:00, 1.81MB/s]


In [3]:
from zipfile import ZipFile
dataset = '/content/llm-detect-ai-generated-text.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
from google.colab.patches import cv2_imshow

In [5]:
train_essay=pd.read_csv('/content/train_essays.csv')
train_essay.head()

,id,prompt_id,text,generated
0,0059830c,0,Cars. Cars have been around since they became ...,0
1,005db917,0,Transportation is a large necessity in most co...,0
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0
3,00940276,0,How often do you ride in a car? Do you drive a...,0
4,00c39458,0,Cars are a wonderful thing. They are perhaps o...,0


In [9]:
train_essay.tail()

,id,prompt_id,text,generated
1373,fe6ff9a5,1,There has been a fuss about the Elector Colleg...,0
1374,ff669174,0,Limiting car usage has many advantages. Such a...,0
1375,ffa247e0,0,There's a new trend that has been developing f...,0
1376,ffc237e9,0,As we all know cars are a big part of our soci...,0
1377,ffe1ca0d,0,Cars have been around since the 1800's and hav...,0


In [6]:
train_essay.shape

(1378, 4)

In [133]:
for i in range(1378):
  if(train_essay['generated'][i]==1):
    break

ESSAY WRITTEN BY AI

In [136]:
train_essay['text'][704]

"This essay will analyze, discuss and prove one reason in favor of keeping the Electoral College in the United States for its presidential elections. One of the reasons to keep the electoral college is that it is better for smaller, more rural states to have more influence as opposed to larger metropolitan areas that have large populations. The electors from these states are granted two votes each. Those from larger, more populated areas are granted just one vote each. Smaller states tend to hold significant power because their two votes for president and vice president add up more than the votes of larger states that have many electors. This is because of the split of the electoral votes. Some argue that electors are not bound to vote for the candidate who won the most votes nationally. They do not have to vote for their own state's nominee unless their state has a winner take all system. However, there are states that have adopted laws that force their electors to vote for their stat

ESSAY WRITTEN BY STUDENT

In [130]:
train_essay['text'][0]

'Cars. Cars have been around since they became famous in the 1900s, when Henry Ford created and built the first ModelT. Cars have played a major role in our every day lives since then. But now, people are starting to question if limiting car usage would be a good thing. To me, limiting the use of cars might be a good thing to do.\n\nIn like matter of this, article, "In German Suburb, Life Goes On Without Cars," by Elizabeth Rosenthal states, how automobiles are the linchpin of suburbs, where middle class families from either Shanghai or Chicago tend to make their homes. Experts say how this is a huge impediment to current efforts to reduce greenhouse gas emissions from tailpipe. Passenger cars are responsible for 12 percent of greenhouse gas emissions in Europe...and up to 50 percent in some carintensive areas in the United States. Cars are the main reason for the greenhouse gas emissions because of a lot of people driving them around all the time getting where they need to go. Article

In [7]:
train_prompt=pd.read_csv('/content/train_prompts.csv')
train_prompt.head()

,prompt_id,prompt_name,instructions,source_text
0,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
1,1,Does the electoral college work?,Write a letter to your state senator in which ...,# What Is the Electoral College? by the Office...


In [105]:
x=train_essay['text']
y=train_essay['generated']

In [106]:
print(x.shape,y.shape)

(1378,) (1378,)


In [107]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming x is a column of text data
vectorizer = TfidfVectorizer(min_df=1, stop_words='english', lowercase=True)
x_vectorized = vectorizer.fit_transform(x)

In [109]:
print(x_vectorized.shape,y.shape)

(1378, 16066) (1378,)


In [110]:
from imblearn.over_sampling import RandomOverSampler
os=RandomOverSampler()
x_res,y_res=os.fit_resample(x_vectorized,y)

In [111]:
print(x_vectorized.shape,y.shape)

(1378, 16066) (1378,)


In [112]:
print(x_res.shape,y_res.shape)

(2750, 16066) (2750,)


In [113]:
x1=np.array(x_res)

In [114]:
type(x1)

numpy.ndarray

In [115]:
type(y_res)

pandas.core.series.Series

In [116]:
y1=np.array(y_res)

In [ ]:
print(x_res)

In [124]:
from sklearn.linear_model import LogisticRegression
model1=LogisticRegression()
model1.fit(x_res,y_res)

LogisticRegression()

In [127]:
y_pred=model1.predict(x_res)

In [128]:
print(y_pred)

[0 0 0 ... 1 1 1]


In [141]:
from sklearn.metrics import accuracy_score
ac=accuracy_score(y_pred,y_res)
ac=ac*100
print(f'The accuracy of the model is : {ac} %')

The accuracy of the model is : 100.0 %


In [137]:
input_text=["This essay will analyze, discuss and prove one reason in favor of keeping the Electoral College in the United States for its presidential elections. One of the reasons to keep the electoral college is that it is better for smaller, more rural states to have more influence as opposed to larger metropolitan areas that have large populations. The electors from these states are granted two votes each. Those from larger, more populated areas are granted just one vote each. Smaller states tend to hold significant power because their two votes for president and vice president add up more than the votes of larger states that have many electors. This is because of the split of the electoral votes. Some argue that electors are not bound to vote for the candidate who won the most votes nationally. They do not have to vote for their own state's nominee unless their state has a winner take all system. However, there are states that have adopted laws that force their electors to vote for their state's candidate. It seems that, no matter how, electors are not bound to vote for the candidate who won the most nationally. This is not always the case because of state legislatures who can overrule the electors and vote for the alternative candidate their citizens have selected for them, even if the voter lives in a state without a winner take all system."]
input_text_feature_extraction=vectorizer.transform(input_text)
prediction=model1.predict(input_text_feature_extraction)
if(prediction[0]==0):
  print('The essay was written by STUDENT')
elif(prediction[0]==1):
  print('The essay was written by AI')

The essay was written by AI
